In [2]:
import numpy as np
import json_tricks


inputs = json_tricks.load('inputs/inputs.json')
answer = {}

# Contra- and Co- Variant Coordinates (Task)

Find convariant and contravariant coordinates of the vectors:

$$\mathbf x = \begin{bmatrix}
1 \\ 2 \\ 3
\end{bmatrix}, 

\mathbf y = \begin{bmatrix}
3 \\ 4 \\ 5
\end{bmatrix}$$

in two sets of basis:

$$
\begin{bmatrix}
\mathbf a_1 & \mathbf a_2 & \mathbf a_3
\end{bmatrix}
=
\begin{bmatrix}
1 & 1 & 1 \\
0 & 1 & 1 \\
0 & 0 & 1 \\
\end{bmatrix}
$$

$$
\begin{bmatrix}
\mathbf b_1 & \mathbf b_2 & \mathbf b_3
\end{bmatrix}
=
\begin{bmatrix}
1 & 0 & 0 \\
1 & 1 & 0 \\
1 & 1 & 1 \\
\end{bmatrix}
$$

In [8]:
answer['task0'] = {
    'a': {
        'contravariant_x': np.array([-1, -1, 3]),
        'contravariant_y': np.array([-1, -1, 5]),
        'covariant_x': np.array([1, 3, 6]),
        'covariant_y': np.array([3, 7, 12])
    },
    'b': {
        'contravariant_x': np.array([1, 1, 1]),
        'contravariant_y': np.array([3, 1, 1]),
        'covariant_x': np.array([6, 5, 3]),
        'covariant_y': np.array([12, 9, 5])
    }
}

# 3-dimensional arrays

It is time to start working with 3 dimensional arrays. That
might be useful when you want to speed up some calculations.

## Task

You are given a set of $N$ basis vectors in the form of row-matrix $B$ of size $N \times M$ and a set of **contravariant** coordinates in that basis $C$ for $K$ vectors of size $N \times K$:

$$B = 
\begin{bmatrix}
- & \mathbf b_1 & - \\
- & \mathbf b_2 & - \\
  & \dots & \\
- & \mathbf b_N & -
\end{bmatrix}$$

$$C =
\begin{bmatrix}
c_1^1 & c_1^2 & \dots & c_1^K \\
c_2^1 & c_2^2 & \dots & c_2^K \\
\vdots & \vdots & \ddots & \vdots \\
c_N^1 & c_N^2 & \dots & c_N^K \\
\end{bmatrix}
$$

Thus, $N$ is the number of the basis
vectors and, thus number of coordinates in the span,
$M$ is the dimensionality of the space, $K$ is the number of vectors to reconstruct.

**The task is** to reconstruct the vectors that are defined using these coordinates. The answer should be
a matrix of shape $K \times M$

**You should not use the cycles in this task**

# Interface

## Inputs

`B, C` -- 2D `numpy` arrays

## Outputs

`V` -- 2D `numpy` array


<details>
<summary> <b>Hint</b> </summary>
So, how are you supposed to deal with this task?

If you are allowed to use cycles, that is rather simple: you
iterate over the basis vectors and corresponding coordinates,
multiply one with another and collect the result one-by-one.

But the trick here is that you should do that in a vectorized
manner. So, how to deal with it?

We want in the end to recieve an array of the size 
$K \times M$.

And here is how this get it:
1) reshape your basis vectors to the size of 
$N \times 1 \times M$. It is non-trivial to do that with
usual `reshape`, so there is a function 
`numpy.expand_dims(x, axis)` that allows to add a
dimension of an array to the specified axis (i.e., the 
position in which the new dimension should be inserted). In this
case the `axis=1` as we want to insert dimension of
size `1` to the first position.
2) reshape the coordinates in this basis to the size of
$N \times K \times 1$. Use `numpy.expand_dims` yet again.
3) multiply two tensors from above and receive array of size
$N \times K \times M$. This will first cast the shapes of 
the abovementioned arrays and then perform the multiplication.
This result holds all the required products you may ever need.
4) Now sum this array along the `axis=0` to get $K \times M$
array.
5) Enjoy

Note that although this approach works really fast, it
requires quite a lot of memory. There is another way of doing
this operation that we will study later.
</details>

In [11]:
import numpy as np

def reconstruct_vectors(B, C):
    B_exp = np.expand_dims(B, axis=1)
    C_exp = np.expand_dims(C, axis=2)
    mult = B_exp * C_exp
    
    V = np.sum(mult, axis=0)
    
    return V.T


In [12]:
answer['task1'] = []
for input in inputs['task1']:
    res = reconstruct_vectors(**input)
    answer['task1'].append(res)

# Task

You are given a set of vectors as basis that are stored
in a row-matrix `numpy.ndarray` called `B` and a set of vectors
`X`. It is guaranteed that the vectors from `X` come from
the span of the `B` vectors. Find **covariant** coordinates
of vectors `X` in this `basis`.

## Interface

**Inputs**: `basis: numpy.ndarray`

**Outputs**: `coords: numpy.ndarray`

In [63]:
def get_covariant_coordinates(B, X):

    B = np.array(B)
    X = np.array(X)

    pseudo_inv = np.linalg.pinv(B.T)

    coords = pseudo_inv @ X
    return coords.astype(np.int64)


In [64]:
answer['task2'] = []
for input in inputs['task2']:
    res = get_covariant_coordinates(**input)
    answer['task2'].append(res)

# Task

For the basis vectors and contravariant coordinates from task #1, find covariant coordinates

In [29]:
def contravariant_to_covariant(B, C):
    print(B.shape, C.shape)
    G = B @ B.T     
    res = G @ C      
    return res


In [30]:
answer['task3'] = []
for input in inputs['task1']:
    res = contravariant_to_covariant(**input)
    answer['task3'].append(res)

(8, 9) (8, 6)
(5, 9) (5, 9)
(5, 5) (5, 2)
(9, 9) (9, 7)
(5, 5) (5, 7)
(5, 6) (5, 2)
(4, 5) (4, 8)
(9, 9) (9, 6)
(7, 9) (7, 4)
(3, 4) (3, 4)
(4, 9) (4, 2)
(8, 9) (8, 7)
(6, 6) (6, 3)
(8, 9) (8, 3)
(3, 3) (3, 6)
(7, 8) (7, 8)
(6, 7) (6, 6)
(2, 3) (2, 5)
(6, 9) (6, 6)
(3, 7) (3, 5)
(5, 5) (5, 2)
(5, 5) (5, 9)
(9, 9) (9, 6)
(7, 9) (7, 9)
(5, 7) (5, 9)
(5, 6) (5, 7)
(7, 8) (7, 9)
(8, 8) (8, 5)
(2, 2) (2, 2)
(9, 9) (9, 9)
(7, 8) (7, 7)
(7, 9) (7, 9)
(2, 5) (2, 6)
(5, 5) (5, 8)
(9, 9) (9, 7)
(3, 5) (3, 7)
(8, 8) (8, 9)
(9, 9) (9, 2)
(9, 9) (9, 8)
(7, 7) (7, 4)
(3, 3) (3, 3)
(6, 8) (6, 7)
(6, 8) (6, 3)
(2, 6) (2, 2)
(8, 8) (8, 6)
(9, 9) (9, 5)
(5, 9) (5, 9)
(8, 8) (8, 6)
(7, 7) (7, 4)
(6, 8) (6, 3)
(8, 8) (8, 9)
(9, 9) (9, 5)
(4, 8) (4, 8)
(6, 9) (6, 4)
(8, 9) (8, 8)
(6, 9) (6, 8)
(6, 6) (6, 7)
(5, 8) (5, 2)
(4, 7) (4, 9)
(5, 8) (5, 6)
(8, 8) (8, 5)
(5, 7) (5, 8)
(6, 6) (6, 8)
(2, 5) (2, 7)
(5, 5) (5, 8)
(8, 8) (8, 9)
(6, 6) (6, 3)
(3, 6) (3, 9)
(8, 8) (8, 4)
(2, 9) (2, 5)
(8, 9) (8, 3)
(8, 8)

In [ ]:
json_tricks.dump(answer, '.answer.json')